In [ ]:
import os
import time
import google.generativeai as genai
import ast
import re
import pandas as pd
from tqdm import tqdm
import time
import csv

In [ ]:
data = pd.read_csv('/content/main_dataframe.csv')

df = data[data["BIRADS Score"] == 1]

df.head(3)

,File Name,BIRADS Score,Content
4,1389.txt,1,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nMeme parankim...
29,37.txt,1,BİLATERAL MAMOGRAFİ İNCELEMESİ\nBilateral meme...
37,1390.txt,1,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nMeme parankim...


In [ ]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    }
]

In [ ]:
genai.configure(api_key="WRİTE_YOUR_OWN_API_KEY") # Add your API_KEY not mine

model = genai.GenerativeModel('gemini-1.5-flash', safety_settings=safety_settings)

In [ ]:
mistake_sentences = []

In [ ]:
def submission_prediction(sentence: str):
    global mistake_sentences

    response = model.generate_content("""

    Sen bir yapay zeka modelisin ve verilen Türkçe memeografi raporunu genel anlam ve kelime anlamı açısından analiz eden bir sentetik veri üreticisisin.

    Bu rapor, BIRADS 1 kategorisine aittir ve normal, herhangi bir malignite belirtisi olmayan bulgular içermektedir.

    Aşağıda verilen raporu analiz et ve anla. Anlamını anladıktan sonra, bu raporu hazırlayan bir doktorun aynı BIRADS 1 kategorisinde benzer raporları nasıl yazmış olabileceğini düşün. Bu doğrultuda, orijinal raporun uzunluğuna yakın ve tıbbi doğruluğa sahip 4 adet yeni rapor üret.

    Sadece aşağıdaki Python liste formatında yaz. Başka hiçbir şey yazma:
    Çıktı formatı: ["<rapor1>", "<rapor2>", "<rapor3>", "<rapor4>"]

    \n\n
    Orijinal Rapor: """ + sentence)

    try:
        pattern = re.compile(r'\[.*?\]', re.DOTALL)
        # print(response.text)
        match = pattern.search(response.text)
        if match:
            response_json = ast.literal_eval(match.group(0))
            return response_json

        else:
            mistake_sentences.append(sentence)
            print("It could not be augmented.")
            return []

    except Exception as err:
        mistake_sentences.append(sentence)
        print("It could not be augmented. Err:", err)
        return []



In [ ]:
birads_augmented_texts = []

In [ ]:
birads_texts =  df['Content'].values

for text in tqdm(birads_texts):
    if text not in birads_augmented_texts:
        birads_augmented_texts.append(text)

        augmented_text_list = submission_prediction(sentence=text)

        birads_augmented_texts.extend(augmented_text_list)
        time.sleep(4.1)

 12%|█▏        | 6/51 [01:02<07:36, 10.15s/it]

It could not be augmented. Err: unterminated string literal (detected at line 1) (<unknown>, line 1)


100%|██████████| 51/51 [09:02<00:00, 10.63s/it]


In [ ]:
augmented_df = pd.DataFrame(birads_augmented_texts)

augmented_df

,0
0,BİLATERAL MAMOGRAFİ İNCELEMESİ:\nMeme parankim...
1,BİLATERAL MAMOGRAFİ İNCELEMESİ: \nMeme paranki...
2,BİLATERAL MAMOGRAFİ İNCELEMESİ: \nMeme paranki...
3,BİLATERAL MAMOGRAFİ İNCELEMESİ: \nMeme paranki...
4,BİLATERAL MAMOGRAFİ İNCELEMESİ: \nMeme paranki...
...,...
246,BİLATERAL MAMOGRAFİ İNCELEMESİ\nCilt-cilt altı...
247,BİLATERAL MAMOGRAFİ İNCELEMESİ\nCilt-cilt altı...
248,BİLATERAL MAMOGRAFİ İNCELEMESİ\nCilt-cilt altı...
249,BİLATERAL MAMOGRAFİ İNCELEMESİ\nCilt-cilt altı...


In [ ]:
augmented_df.to_csv('augmented_birads_1.csv', index=False)